# 네이버 뉴스 크롤링

### 함수 만들어서 호출해보기

In [4]:
import json  # JSON 파일 저장을 위한 모듈

def main():  # 메인 함수 정의
    node = 'news'  # [CODE 0] 네이버 오픈 API 검색 카테고리 설정 (news = 뉴스 검색)
    
    srcText = input('검색어를 입력하세요: ')  # [CODE 1] 사용자로부터 검색어 입력 받기
    cnt = 0  # 수집된 뉴스 개수 카운트
    jsonResult = []  # 뉴스 데이터를 저장할 리스트

    # [CODE 2] 네이버 뉴스 검색 API 호출: 1페이지(start=1), 최대 100개 가져오기
    jsonResponse = getNaverSearch(node, srcText, 1, 100)
    total = jsonResponse['total']  # 전체 검색 결과 수 추출

    # [반복] 검색 결과가 존재하고, 한 페이지에 표시된 결과 수가 0이 아닐 때 반복
    while ((jsonResponse is not None) and (jsonResponse['display'] != 0)):
        for post in jsonResponse['items']:  # 검색 결과 중 각 뉴스 항목 반복
            cnt += 1  # 뉴스 건수 카운트 증가
            getPostData(post, jsonResult, cnt)  # [CODE 3] 뉴스 항목을 파싱하여 리스트에 저장

        # 다음 검색을 위한 시작 위치 계산
        start = jsonResponse['start'] + jsonResponse['display']

        # 다음 페이지 뉴스 데이터 요청
        jsonResponse = getNaverSearch(node, srcText, start, 100)

    # 전체 검색 결과 수 출력
    print('전체 검색 : %d 건' % total)

    # 수집한 데이터를 JSON 파일로 저장
    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        # JSON 파일 문자열로 변환 (정렬 및 인코딩 옵션 포함)
        jsonFile = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)
        outfile.write(jsonFile)  # 파일로 저장

    # 수집 완료 메시지 출력
    print("가져온 데이터 : %d 건" % cnt)
    print('%s_naver_%s.json SAVED' % (srcText, node))

---

### 내 방식대로 해보기

### 1. 요청 URL만들기

In [40]:
# 필요한 라이브러리 불러오기
import urllib.request    # URL 요청을 보내기 위한 모듈
import urllib.parse      # 검색어(한글 등)를 URL 인코딩할 때 필요
import json              # JSON 데이터를 다루기 위한 모듈

In [41]:
# [반드시 입력] 네이버 개발자 센터에서 발급받은 값 입력
client_id = "YofuzhxNNw_eo4yEXbea"  # 네이버 애플리케이션의 클라이언트 ID
client_secret = "XbvYKiWU37"        # 네이버 애플리케이션의 클라이언트 ID

In [42]:
# 검색 설정
query = input('검색어 입력 : ') # 검색어 입력
node = input('검색 API 대상 입력 : ex) news, blog, book') # # 검색 API 대상: news, blog, book 등
start = 1                    # 검색 시작 위치 (1부터 시작)
display = 10                 # 한 번에 가져올 대상(ex.뉴스)의 개수 (최대 100개까지 가능)

# 검색어를 URL 인코딩하고, 전체 요청 URL 구성
url = f"https://openapi.naver.com/v1/search/{node}.json?query={urllib.parse.quote(query)}&start={start}&display={display}"

In [43]:
# API 요청을 위한 Request 객체 생성
request = urllib.request.Request(url)

In [44]:
# 요청 헤더에 클라이언트 ID와 시크릿 추가 (인증용)
request.add_header("X-Naver-Client-Id", client_id)
request.add_header("X-Naver-Client-Secret", client_secret)

In [45]:
# 요청 내보내고 응답 받기
response = urllib.request.urlopen(request)
rescode = response.getcode() # 응답 코드 확인 (200이면 성공)
print(rescode)
# 꼭 200일때만 다음으로 넘어가기!
# 아닐 경우 예외처리 안해둬서 인증 실패 시 노트북 중단돰.

200


In [46]:
# 응답 본문(바이트)을 문자열로 디코딩하고 JSON으로 파싱
result = json.loads(response.read().decode('utf-8'))

### 3. 크롤링 결과 출력

In [47]:
# 받아온 뉴스 데이터 출력
for idx, item in enumerate(result['items'], 1):
    print(f"[{idx}] {item['title']}") # node 제목 출력 (HTML 태그 포함 될 수 있음)
    print(item['description'])        # node 요약 설명
    print(item['link'])               # node 링크
    print()                           # 줄바꿈

[1] <b>영화</b> &lt;<b>야당</b>&gt; 관람 <b>후기</b> - 배우들의 하드캐리로 끝까지 몰입한... 
시사회 <b>후기</b>가 꽤 괜찮다는 소식에 결국 극장을 찾게 되었어요! 사진 출처 : &lt;<b>야당</b>&gt; 공식 홈페이지 &lt;<b>야당</b>&gt;은 대한민국 마약 수사의 이면을 다룬 범죄 액션 <b>영화</b>로~ 수사기관과 거래하는 브로커, 이른바 '<b>야당</b>... 
https://blog.naver.com/2_children/223848984044

[2] 끝까지 쫄깃했던 <b>영화</b> '<b>야당</b>' <b>후기</b>(스포 없음)
안녕하세요 오래간만에 <b>영화</b>관에서 재미있게 관람한 <b>영화</b> '<b>야당</b>' <b>후기</b>로 돌아왔어요. 스포 없습니다. 먼저 오리지널 티켓. 강하늘, 유해진이 나온다는 정도만 알고 정말 아~~무 정보도 없이 갔는데요. <b>영화</b>... 
https://blog.naver.com/pinkatena-/223848459516

[3] <b>영화</b> <b>야당</b> 관람평 평점 상영일정 <b>후기</b> 리뷰 쿠키
<b>영화</b> <b>야당</b> <b>후기</b>에서 먼저 말씀드리고 싶은 건 수위는 보통인데(관람등급이 청소년관람불가인 걸 감안하고) 그와 좀 다른 측면에서 예를 들면 차가 일부러 사람을 친 뒤에 굳이 아무렇지 않게 다리를 한번 더... 
https://blog.naver.com/hyunjee1110/223839052093

[4] <b>영화</b> <b>야당</b> 정보와 <b>후기</b>, 출연진, 관람평, 뜻, 쿠키 - 한국 범죄 액션
내부자들은 재벌과 권력가에게 이용만 당하고 버려진 거물 깡패가, 하급 검사와 손잡고 높으신 분들께 복수하는 짜릿한 이야기를 그린 작품입니다. 한국 범죄 액션 <b>영화</b> <b>야당</b> <b>후기</b>, 정보, 관람평을 마치겠습니다.
https://blog.naver.com/